In [ ]:
!pip install ../input/faiss-163/faiss_cpu-1.6.3-cp37-cp37m-manylinux2010_x86_64.whl

In [ ]:
# load gallery features

import faiss
import numpy as np
import sys
# sys.path.insert(2, '/kaggle/working/input/cassavacodes/Cassava/classification')
sys.path.insert(0, '/kaggle/working/input/cassavacodes/Cassava/retrieval')

In [ ]:
# write image filenames in a txt

import os

test_img_lst = os.listdir('../input/cassava-leaf-disease-classification/test_images')

f = open('test_images.txt', 'w')
for img_file in test_img_lst:
    f.write('/kaggle/input/cassava-leaf-disease-classification/test_images/' + img_file + ',5\n')
f.close()

# os.path.exists('/kaggle/working/test_images.txt')
# f=open('test_images.txt')
# f.readlines()

In [ ]:
!mkdir ./log
!mkdir ./features

In [ ]:
# 抽特征
try:
    !cd ../input; python -m cassavacodes.Cassava.retrieval.extract.run --config-file ./configs/extract_r50_gq_infer.py --load-path ../input/wwwwww/weight_epoch_13.pth --log-dir /kaggle/working/log -device 0
except:
    pass

In [ ]:
!ls /kaggle/working/features/extract_r50_gq_infer/

In [ ]:
np.load('/kaggle/working/features/extract_r50_gq_infer/gallery_targets.npy')

In [ ]:
gallery_features = np.load('../input/gallery/gallery_features.npy')
gallery_targets = np.load('../input/gallery/gallery_targets.npy')

query_features = np.load('/kaggle/working/features/extract_r50_gq_infer/gallery_features.npy')
query_names = np.load('/kaggle/working/features/extract_r50_gq_infer/gallery_names.npy')

faiss.normalize_L2(gallery_features)
faiss.normalize_L2(query_features)



In [ ]:
index = faiss.IndexFlatL2(gallery_features.shape[1])
index.add(gallery_features)

k=3
D, I = index.search(gallery_features, k)

print(gallery_targets[I[-10:]])
outliers = []

for i, indices in enumerate(I):
    if gallery_targets[indices[0]] != gallery_targets[indices[1]]:
        outliers.append(i)
        
        
        
print(gallery_features.shape)
gallery_features = np.delete(gallery_features , outliers, axis=0)
gallery_targets = np.delete(gallery_targets , outliers, axis=0)
print(gallery_features.shape)

In [ ]:
index = faiss.IndexFlatL2(gallery_features.shape[1])
index.add(gallery_features)
k=1
D, I = index.search(query_features, k)


f = open('submission.csv', 'w')
f.write('image_id,label\n')


for fn, i in zip(query_names, I):
    fn = fn.split('/')[-1]
    pred = gallery_targets[i][0]
    f.write(fn + ',' + str(pred) + '\n')
f.close()

In [ ]:
f = open('submission.csv', 'r')
print(f.readlines())
f.close()